# Text Mining

In this project we would explore the methods of preprocessing text which includes:
- BOW (Bag-of-Words)
- CBOW
- TF-IDF
- Word2Vec
- GloVe
- FastText
- OneHotEncoding

## Import Libraries

In [17]:
# Common Python Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Text preprocessing Libraries
import nltk
import re
import string
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer # or LancasterStemmer, RegexpStemmer, SnowballStemmer
from sklearn.feature_extraction.text  import CountVectorizer

# Preprocessing Libraries
from sklearn.model_selection import train_test_split

## Import the data

In [18]:
text_data_path = "./train_data.csv"
text_data = pd.read_csv(text_data_path, sep = ",")

In [19]:
# View the data
text_data.head()

,text,label
0,Here are Thursday's biggest analyst calls: App...,0
1,Buy Las Vegas Sands as travel to Singapore bui...,0
2,"Piper Sandler downgrades DocuSign to sell, cit...",0
3,"Analysts react to Tesla's latest earnings, bre...",0
4,Netflix and its peers are set for a ‘return to...,0


In [20]:
# text_data["label"].value_counts()

## Text Cleaning

In [21]:
def clean_text(text: str, language: str, tokenize: bool = False, remove_stop_words: bool = False, stem_words: bool = False):
    """
    This function is to clean the text from stopwords, punctuation and return a clean text for further analysis

    Args:
        text (str):
            The dataframe containing the text data
        
        language (str):
            This are the available languages:
            - "catalan": "ca"
            - "czech": "cs"
            - "german": "de"
            - "greek": "el"mlaskjdlj
            - "english": "en"
            - "spanish": "es"
            - "finnish": "fi"
            - "french": "fr"
            - "hungarian": "hu"
            - "icelandic": "is"
            - "italian": "it"
            - "latvian": "lv"
            - "dutch": "nl"
            - "polish": "pl"
            - "portuguese": "pt"
            - "romanian": "ro"
            - "russian": "ru"
            - "slovak": "sk"
            - "slovenian": "sl"
            - "swedish": "sv"
            - "tamil": "ta"
        
        tokenize (bool):
            True = return tokenized data
            False = return untokenized data
        
        remove_stop_words (bool):
            True = remove stop words
            False = do not remove stop words

        stem_words (bool):
            True = get the base words (i.e. spraying -> spray)
            False = leave the words as is
    """

    stemmer = PorterStemmer()
    stop_words = set(stopwords.words(language))

    def tokenize_text(text):
        return [w for s in sent_tokenize(text) for w in word_tokenize(s)]
    
    def remove_special_characters(text, characters=string.punctuation.replace('-', '')):
        pattern = re.compile(f"[{re.escape(characters)}]")
        return pattern.sub("", text)

    def stem_text(tokens):
        return [stemmer.stem(t) for t in tokens]

    def remove_stopwords_func(tokens):
        return [w for w in tokens if w not in stop_words]

    # Clean process
    text = text.strip().lower()                          # lowercase + trim
    text = remove_special_characters(text)               # remove punctuation
    tokens = tokenize_text(text)                         # tokenize words

    if remove_stop_words:
        tokens = remove_stopwords_func(tokens)           # remove stopwords
        
    if stem_words:
        tokens = stem_text(tokens)                       # stemming

    if tokenize:
        return tokens                                    # return as tokens
    else:
        return " ".join(tokens)                          # return as string

In [22]:
# Test
sample = "I love the smell of freshly brewed coffee in the morning!"
cleaned = clean_text(sample, language="english", remove_stop_words=True, tokenize=False)
print(cleaned)

love smell freshly brewed coffee morning


## Train Test Split

In [23]:
test_size = 0.20
val_size = 0.10

# Splitting the data into train and temp (which will be further split into validation and test)
train_df, test_df = train_test_split(text_data, test_size=test_size, stratify=text_data['label'], random_state=42) #stratify is used to ensure that the same proportion of each class is present in both the training and test sets

# Splitting train into validation and test sets
train_df, val_df = train_test_split(train_df, test_size=val_size, stratify=train_df['label'], random_state=42)


## Text Preprocessing Methods

### Bag-of-Words

Briefly, the bag-of-words preprocessing method only counts the occurence of each word and does not care about the ordering of the words.

**Example**

Both of these sentences become almost the same for BoW:

- “Coffee is life”
- “Life is coffee”

BoW doesn’t care that the word order is swapped — it just notes that both have “coffee”, “is”, and “life”.

In [28]:
vectorizer = CountVectorizer()
x_train_BOW = vectorizer.fit_transform(train_df["text"])
y_train = train_df["label"]
train_vocab = vectorizer.get_feature_names_out()
print ("train_BOW: ", x_train_BOW.shape)
print ("train_Vocab: ", train_vocab)
print()

x_test_BOW = vectorizer.transform(test_df["text"])
y_test = test_df["label"]
print("test_BOW: ", x_test_BOW.shape)

train_BOW:  (12232, 33542)
train_Vocab:  ['00' '000' '000th' ... 'åkerström' 'є0' 'є1']

test_BOW:  (3398, 33542)
